# Packet Containers

> Loading, unloading and containing packets. Shared logic between Edges and Ports.

In [1]:
#| default_exp packet_container

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Callable, Any
import inspect
from collections import deque
from enum import Enum

import fbdev
from fbdev.packet import Packet
from fbdev.utils import EventHandler, StateHandler, StateView

In [5]:
#|export
__all__ = ['PacketContainer', 'SinglePacketContainer', 'DequePacketContainer']

In [6]:
#|export
FullEmpty = Enum('PacketContainerStatus', ['EMPTY', 'FULL'])
FullEmpty = Enum('PacketContainerStatus', ['NON_EMPTY', 'EMPTY', 'AVAILABLE', 'FULL'])

In [7]:
#|hide
show_doc(fbdev.packet_container.PacketContainer)

---

### PacketContainer

>      PacketContainer ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [8]:
#|export
class PacketContainer(ABC):
    def __init__(self):
        self._empty = StateHandler(True)
        self._full = StateHandler(False)
        self.empty = StateView(self._empty)
        self.full = StateView(self._full)
        
        self.has_changed = EventHandler()
    
    @abstractmethod
    def load(self,
            packet: Packet):
        #TODO proper exception
        if self._full.get(): raise Exception("Port is full.")
        self.has_changed._trigger()
        
    @abstractmethod
    def unload(self):
        #TODO proper exception
        if self._empty.get(): raise Exception("Port is empty.")
        self.has_changed._trigger()
        
    @abstractmethod
    def size(self):
        pass
    

In [9]:
#|hide
show_doc(fbdev.packet_container.SinglePacketContainer)

---

### SinglePacketContainer

>      SinglePacketContainer ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [10]:
#|export
class SinglePacketContainer(PacketContainer):
    def __init__(self):
        super().__init__()
        self._packet = None
       
    def _set_full_state(self):
        self._full.set(True)
        self._empty.set(False)
        
    def _set_empty_state(self):
        self._full.set(False)
        self._empty.set(True)
    
    def load(self,
            packet: Packet):
        super().load(packet)
        self._packet = packet
        self._set_full_state()
        
    def unload(self):
        super().unload()
        packet = self._packet
        self._packet = None
        self._set_empty_state()

        return packet
    
    def size(self):
        if self._full.get():
            return 1
        else:
            return 0

In [11]:
container = SinglePacketContainer()

# Load data
packet = Packet("data")
changed_event1 = container.has_changed.subscribe()
container.load(packet)
assert changed_event1.is_set()
assert container.full.get()
assert container.size() == 1

# Unload data
changed_event2 = container.has_changed.subscribe()
packet_unloaded = container.unload()
assert packet_unloaded.get_data() == "data"
assert changed_event2.is_set()
assert container.empty.get()
assert container.size() == 0

In [12]:
#|hide
show_doc(fbdev.packet_container.DequePacketContainer)

---

### DequePacketContainer

>      DequePacketContainer (maxsize:int=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [13]:
#|export
class DequePacketContainer(PacketContainer):
    def __init__(self, maxsize:int=None):
        super().__init__()
        if maxsize is not None and maxsize < 0:
            #TODO proper exception
            raise Exception("`maxsize` must be larger than 0.")
        self.maxsize = maxsize
        self._num_containers = maxsize if maxsize is not None else 1
        self._containers = [SinglePacketContainer() for i in range(self._num_containers)]
        self._filled_containers = deque(maxlen=self.maxsize)
        self._empty_containers = []
        self._empty_containers.extend(self._containers)
        
    def _update_status_events(self):        
        if len(self._filled_containers) > 0: self._empty.set(False)
        else: self._empty.set(True)
        if len(self._empty_containers) == 0 and self.maxsize is not None: self._full.set(True)
        else: self._full.set(False)

    def __add_more_containers_if_needed(self):
        if self.size() == self._num_containers:
            self._containers.append(SinglePacketContainer())
            self._empty_containers.append(SinglePacketContainer())
            self._num_containers += 1
    
    def load(self, packet: Packet):
        super().load(packet)
        self.__add_more_containers_if_needed()
        cnt = self._empty_containers.pop()
        cnt.load(packet)
        self._filled_containers.append(cnt)
        self._update_status_events()
        
    def load_left(self, packet: Packet):
        super().load(packet)
        self.__add_more_containers_if_needed()
        cnt = self._empty_containers.pop()
        cnt.load(packet)
        self._filled_containers.appendleft(cnt)
        self._update_status_events()
        
    def unload(self):
        super().unload()
        cnt = self._filled_containers.popleft()
        packet = cnt.unload()
        self._empty_containers.append(cnt)
        self._update_status_events()
        return packet
    
    def unload_right(self):
        super().unload()
        cnt = self._filled_containers.pop()
        packet = cnt.unload()
        self._empty_containers.append(cnt)
        self._update_status_events()
        return packet

    def unload_all(self):
        super().unload()
        packets = [cnt.unload() for cnt in self._filled_containers]
        self._empty_containers.extend(self._filled_containers)
        self._filled_containers.clear()
    
    def size(self):
        return len(self._filled_containers)


In [14]:
container = DequePacketContainer(maxsize=3)

# Load data1
changed_event1 = container.has_changed.subscribe()
container.load(Packet("data1"))
assert changed_event1.is_set()

# Load data2
container.load(Packet("data2"))

# Load data3
container.load(Packet("data3"))
assert container.full.get()

# Unload data1
assert container.unload().get_data() == 'data1'
assert container.size() == 2

# Unload data3
assert container.unload_right().get_data() == 'data3'
assert container.size() == 1

# Unload data2
assert container.unload().get_data() == 'data2'
assert container.size() == 0

In [15]:
container = DequePacketContainer()

for i in range(100):
    container.load(Packet(i))

for i in range(100):
    assert container.unload().get_data() == i